In [169]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [170]:
from datetime import datetime, timedelta

In [171]:
df = pd.read_csv('Metro_Interstate_Traffic_Volume.csv', sep=',')

In [172]:
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918


Удаляем колонки согласно ТЗ:

In [173]:
df.drop(columns=['weather_description','weather_main','clouds_all','snow_1h','rain_1h'], axis=1, inplace=True)

In [174]:
df.head()

,holiday,temp,date_time,traffic_volume
0,None,288.28,2012-10-02 09:00:00,5545
1,None,289.36,2012-10-02 10:00:00,4516
2,None,289.58,2012-10-02 11:00:00,4767
3,None,290.13,2012-10-02 12:00:00,5026
4,None,291.14,2012-10-02 13:00:00,4918


In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48204 entries, 0 to 48203
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   holiday         48204 non-null  object 
 1   temp            48204 non-null  float64
 2   date_time       48204 non-null  object 
 3   traffic_volume  48204 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 1.5+ MB


In [176]:
df.shape

(48204, 4)

Убираем дубли, оставляя последнюю запись:

In [177]:
df.drop_duplicates(subset=['date_time'], keep='last', inplace=True)

In [178]:
df.shape

(40575, 4)

In [179]:
unique, counts = np.unique(df['date_time'], return_counts=True)
for key,value in dict(zip(unique, counts)).items():
    if value > 1:
        print(f'Количество записей даты: {key} - {value}')

In [180]:
df['date_time'] =  pd.to_datetime(df['date_time'], format='%Y-%m-%d %H:%M:%S')

In [181]:
df['date_time']

0       2012-10-02 09:00:00
1       2012-10-02 10:00:00
2       2012-10-02 11:00:00
3       2012-10-02 12:00:00
4       2012-10-02 13:00:00
                ...        
48199   2018-09-30 19:00:00
48200   2018-09-30 20:00:00
48201   2018-09-30 21:00:00
48202   2018-09-30 22:00:00
48203   2018-09-30 23:00:00
Name: date_time, Length: 40575, dtype: datetime64[ns]

In [182]:
df.isnull().sum()

holiday           0
temp              0
date_time         0
traffic_volume    0
dtype: int64

In [183]:
df = df.reset_index(drop=True)

In [184]:
for index, row in df.iterrows():
    if index > 0:
        if (df.loc[index, ('date_time')] - df.loc[index - 1, ('date_time')]).seconds > 3600:
            add_counter = int((df.loc[index, ('date_time')] - df.loc[index - 1, ('date_time')]).seconds / 3600) - 1
            for step in range(add_counter):
                line = pd.DataFrame({'holiday': None, 'date_time': df.loc[index - 1, ('date_time')] + timedelta(hours=step + 1)}, index=[df.shape[0] + step])
#                 df = df.append(line, ignore_index=True)
                df = pd.concat([df, line], ignore_index=True)
#                 print(df.loc[index - 1 + 0.1 * (step + 1), ('date_time')])

In [143]:
df.shape

(44395, 4)

In [162]:
df = df.sort_values(by=['date_time']).reset_index(drop=True)

In [163]:
for index, row in df.iterrows():
    if index > 0:
        if (df.loc[index, ('date_time')] - df.loc[index - 1, ('date_time')]).seconds > 3600:
            print(f"{df.loc[index - 1, ('date_time')]} - {df.loc[index, ('date_time')]}")

In [168]:
df.loc[22]

holiday                          None
temp                              NaN
date_time         2012-10-03 07:00:00
traffic_volume                    NaN
Name: 22, dtype: object